In [1]:
import codecs
import re
import pymorphy2
import math
import operator
import collections
from collections import Counter
import nltk
from nltk import RegexpTokenizer
def tokenize1(text):    
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(text)

In [2]:
f = codecs.open('SentiRuEval_rest_markup.xml', 'r', 'utf-8')
rest = f.read()
f.close()
morph = pymorphy2.MorphAnalyzer()
texts = re.findall('<text>(.*?)</text>', rest)

In [3]:
for i in texts:
    print(i)


День 8-го марта прошёл, можно и итоги подвести. Решил написать отзыв о ресторане в котором отметили прекрасный весений праздник, прочитал отзывы edik077 и Rules77777и понял что либо мы были вразных ресторанах, либо у ребят что-то незаладилось. Но теперь о ресторане. Столик бронировали заранее и сделали так как предложил администратор т.е. сделали предварительный заказ, когда придя увидели полностью заполненый ресторан поняли что совет нам дали действительно правильный, в ресторане было человек 70-80, тут действительно горячее блюдо можно ждать весьма долго. Меню достаточно разнообразное и весьма вкусное, мне и моим друзьям понравилось всё что нам принесли, а принесли нам немало. Обслуживание может и не самое лучьшее в городе, но официанты делали всё что нужно. Должен сказать что ждать нам всёравно пришлось, не очень долго конечно, но всё-же. Интерьер хороший, удобные диваны.Очень хорошая музыкальная программа и весёлый ведущий. В общем я остался доволен своим выбором, и тем как мы пров

Удаление стоп-слов, лемматизация

In [4]:
corpus = []
stopwords = nltk.corpus.stopwords.words('russian')
for text in texts:    
    text = text.lower()
    words = tokenize1(text)
    res = [morph.parse(word)[0].normal_form for word in words if word not in stopwords]
    corpus.append(res)
      

In [5]:
print(corpus)

[['день', '8', 'го', 'март', 'пройти', 'итог', 'подвести', 'решить', 'написать', 'отзыв', 'ресторан', 'который', 'отметить', 'прекрасный', 'весения', 'праздник', 'прочитать', 'отзыв', 'edik077', 'rules77777ь', 'понять', 'либо', 'вразный', 'ресторан', 'либо', 'ребята', 'незаладиться', 'ресторан', 'столик', 'бронировать', 'заранее', 'сделать', 'предложить', 'администратор', 'том', 'е', 'сделать', 'предварительный', 'заказ', 'прийти', 'увидеть', 'полностью', 'заполненый', 'ресторан', 'понять', 'совет', 'мы', 'дать', 'действительно', 'правильный', 'ресторан', 'человек', '70', '80', 'действительно', 'горячее', 'блюдо', 'ждать', 'весьма', 'долго', 'меню', 'достаточно', 'разнообразный', 'весьма', 'вкусный', 'мой', 'друг', 'понравиться', 'весь', 'мы', 'принести', 'принести', 'мы', 'немало', 'обслуживание', 'самый', 'лучьшея', 'город', 'официант', 'делать', 'весь', 'нужно', 'должный', 'сказать', 'ждать', 'мы', 'всёравно', 'прийтись', 'очень', 'долго', 'весь', 'интерьер', 'хороший', 'удобный', '

In [6]:
def compute_tfidf(corpus):
    def compute_tf(text):
        tf_text = Counter(text)
        for i in tf_text:
            tf_text[i] = tf_text[i]/float(len(tf_text))
        return tf_text

    def compute_idf(word, corpus):
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

    documents_list = []
    for text in corpus:
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, corpus)
        documents_list.append(sorted(tf_idf_dictionary.items(), key=operator.itemgetter(1)))
    return documents_list

In [7]:
a = compute_tfidf(corpus)

tf-idf для нашего корпуса

In [8]:
a

[[('очень', 0.0023671281519575768),
  ('обслуживание', 0.0028577371987082286),
  ('интерьер', 0.00294421806166967),
  ('понравиться', 0.003213706275675767),
  ('кухня', 0.003213706275675767),
  ('блюдо', 0.0033545159325590757),
  ('вкусный', 0.004130300699402312),
  ('официант', 0.004130300699402312),
  ('меню', 0.004360956625054273),
  ('столик', 0.004480702497290398),
  ('спасибо', 0.004603599935793429),
  ('уютный', 0.004794231838593768),
  ('человек', 0.005061152093701641),
  ('день', 0.005271856997608961),
  ('сказать', 0.00534425896168389),
  ('персонал', 0.005492520085185322),
  ('остаться', 0.005724129524498331),
  ('заказ', 0.005885174212344059),
  ('который', 0.006051971698131668),
  ('друг', 0.006051971698131668),
  ('довольный', 0.006051971698131668),
  ('свой', 0.006591397606365326),
  ('выбор', 0.006591397606365326),
  ('общий', 0.006687686237041954),
  ('вечер', 0.006786002144997543),
  ('решить', 0.007778628610792245),
  ('самый', 0.007903164874768751),
  ('отметить', 0

Напечатаем слова с самыми высокими показателями

In [9]:
for i in a:
    print(i[-1], i[-2], i[-3])

('либо', 0.034499485012569255) ('весьма', 0.03135694661088671) ('всёравно', 0.024765549004521388)
('бахра', 0.05617551359562169) ('сотрудник', 0.04883331857942702) ('руководство', 0.04453840982197138)
('тарелка', 0.0229626395464121) ('встреча', 0.021934204032214053) ('ломтик', 0.018725171198540565)
('крапива', 0.07551462483345867) ('помогать', 0.059871305006584476) ('посещение', 0.03899189197093941)
('мастер', 0.0548380013671545) ('бивштекс', 0.0548380013671545) ('гостеприимный', 0.04347797149287682)
('свинина', 0.05487535697072666) ('утка', 0.04305733466143028) ('европейский', 0.032796662516921406)
('пабовский', 0.04305039359664465) ('затем', 0.03413223930281918) ('весьма', 0.027254168549649196)
('абсолютно', 0.04800058083815153) ('слух', 0.045160707008244885) ('дружный', 0.045160707008244885)
('отлиный', 0.07197487679439028) ('минут30', 0.07197487679439028) ('посадить', 0.053160502065391455)
('квартира', 0.03838660095700815) ('ночь', 0.03336943436260846) ('quot', 0.028540114879876018

Видно, что попалось много мусора (в основном названия блюд, не связанные с оценкой глаголы, цены и т.д.). Посмотрим только на прилагательные.

In [10]:
d = {}
for i in a:
    for k in i:
        if k[0].endswith('ый') or k[0].endswith('ий'):
            d[k[0]] = k[1]
            
        

In [11]:
for i in sorted(d, key=d.get, reverse=True):
    print (i + ' : ' + str(d[i]))
    

детский : 0.11158864441733013
отлиный : 0.07197487679439028
предложный : 0.069793819921833
дмитрий : 0.06397766826168025
мороженный : 0.06397766826168025
резиновый : 0.06397766826168025
новогодний : 0.06397766826168025
оперативный : 0.06224854209244565
круглый : 0.06160510959250794
классичсеский : 0.057579901435512226
фруктовый : 0.05561572393768077
взрослый : 0.05485386470565774
незабываемый : 0.0548380013671545
присущий : 0.0548380013671545
хрустальный : 0.0548380013671545
шикарный : 0.05372008629765874
лаконичный : 0.052345364941374754
любезный : 0.05068774839889893
гуманный : 0.05006947950914106
привередливый : 0.05006947950914106
бравый : 0.04848833805095766
грушевый : 0.047983251196260186
невкусный : 0.04767062051801209
хвалебный : 0.046529213281222004
каминный : 0.046529213281222004
синий : 0.046063921148409785
дружеский : 0.045503774130829726
отзывчивый : 0.045503774130829726
услужливый : 0.045503774130829726
дружный : 0.045160707008244885
острый : 0.04453840982197138
розовый :

In [12]:
w = codecs.open('result_tfidf_adj.txt', 'w', 'utf-8')
for i in sorted(d, key=d.get, reverse=True):
    w.write (i + ' : ' + str(d[i])+ '\r\n')
w.close()
    

Тут тоже есть мусор, поэтому придётся многое отсеять руками.